In [103]:
import requests
import json
import pandas as pd
import re


In [104]:
# 门类列表
url_getMl = 'https://yz.chsi.com.cn/zsml/pages/getMl.jsp'

# 专业列表
url_getZy = 'https://yz.chsi.com.cn/zsml/pages/getZy.jsp'

# 省市列表
url_getSs = 'https://yz.chsi.com.cn/zsml/pages/getMl.jsp'
url_zydo = 'https://yz.chsi.com.cn/zsml/code/zy.do'
s = requests.session()
a = s.get(url_getZy)
a = json.loads(a.text)

In [105]:
fxdm = pd.DataFrame(a)
fxdm

,mc,dm
0,哲学,0101
1,应用伦理,0151
2,理论经济学,0201
3,应用经济学,0202
4,金融,0251
...,...,...
227,智能科学与技术,1405
228,纳米科学与工程,1406
229,区域国别学,1407
230,文物,1451


In [106]:
burl = "https://yz.chsi.com.cn/zsml/queryAction.do"

zy_url = 'https://yz.chsi.com.cn/zsml/code/zy.do'
host = "http://yz.chsi.com.cn"

f2 = {'ssdm': '',
'dwmc': '',
'mldm':'' ,
'mlmc': '',
'yjxkdm': '0702',
'zymc': '',
'xxfs': '1',
'pageno': 1
}
head = {
            "User-Agent":
            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKi"
            "t/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"
        }

In [107]:
def get_school_url(ssdm='',dwmc='',mldm='',mlmc='',yjxkdm='',zymc='',xxfs='1'):
    '''
    获取学校网址
    :param ssdm:    省市代码
    :param dwmc:    单位名称
    :param mldm:    门类代码
    :param mlmc:    门类名称
    :param yjxkdm:  研究学科代码
    :param zymc:    专业名称
    :param xxfs:    学习方向
    '''
    
    url = "https://yz.chsi.com.cn/zsml/queryAction.do"
    f = {'ssdm': ssdm,
        'dwmc': dwmc,
        'mldm':mldm ,
        'mlmc': mlmc,
        'yjxkdm': yjxkdm,
        'zymc': zymc,
        'xxfs': xxfs,
        'pageno': 1
        }
    head = {
            "User-Agent":
            "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKi"
            "t/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36"
        }
    response = requests.post(url,f,headers=head)
    html1 = response.text
    html = html1
    schools_url = []
    while (html!=html1) or (f['pageno']==1):
        reg = re.compile(r'(<tr>.*? </tr>)', re.S)
        content = re.findall(reg, html)
        schools_url+=re.findall(r'<a href="(.*?)" target="_blank">\((.*?)\)(.*?)</a>',
                                    str(content))
        f['pageno'] +=1
        response = requests.post(url,f,headers=head)
        html = response.text
    return schools_url

In [108]:
def get_data(school_url):
    pageno = 1
    # school_name = unquote(re.findall('dwmc=(.*?)&',school_url)[0])
    html = ''
    response = requests.post(school_url+'&pageno='+str(pageno))
    html1 = response.text
    html = html1
    school_data = []
    while (html!=html1) or (pageno==1):

        reg = re.compile(r'(<tr>.*? </tr>)', re.S)
        content = re.findall(reg, html)
        
        for c in content[1:]:
            zy_data = re.findall(r'<td.*?>(.*?)</td>',c)
            zs_details = re.findall("cutString\(\'(.*?)\'",c)
            nzrs = zs_details[0].split('：')[1]
            details_url = re.findall('<a href="(.*?)" target="_blank">',c)[0]
            
            response = requests.post(host+details_url)
            zy_html = response.text.replace('\n','').replace('\r','').replace(' ','')
            w = re.findall('<td.*?>(.*?)</td>',zy_html)
            bksj = []
            for d in w[-4:]:
                
                ksdata = re.findall('(.*?)<spanclass="sub-msg">(.*?)</span>',d)[0]
                bksj.append(ksdata[1])
                zy_data.append(ksdata[0])

            bksj = list(set(bksj))
            bksj = ','.join(bksj)
            zy_data.append(bksj)
            xxgw_url =re.findall('(https://.*?/)',str(w[-4:]))[0] if re.findall('(https://.*?/)',str(w[-4:])) else ''
            zy_data.append(nzrs)
            zy_data.append(xxgw_url)
            zy_data.append(zs_details[1])
            school_data.append(zy_data)
        pageno+=1
        response = requests.post(school_url+'&pageno='+str(pageno))
        html = response.text
    return school_data
# 考试方式、院系所、专业、研究方向、学习方式、指导教师、拟招人数、学校官网、备注

In [109]:
page = requests.get('https://static.kaoyan.cn/json/school/code.json')
a = json.loads(page.text)
schools_id = {}
for c in a['data']:
    d = a['data'][c]
    schools_id.update({d['n']:d['id']})


In [115]:
yjxkdm = '1202'
schools_url = get_school_url(yjxkdm=yjxkdm)


fxmc = ''.join(list(fxdm[fxdm['dm']==yjxkdm][['mc','dm']].values[0][::-1]))
pj_data = pd.read_excel('第五轮学科评估(部分)+第四轮学科评估结果Excel表格.xlsx')
zy_data = []
for s in schools_url:
    school_data = get_data(host+s[0])
    pj = pj_data[(pj_data['Unnamed: 9']==s[2]) & (pj_data['Unnamed: 1']==fxmc)][['Unnamed: 2']].values
    xx = pj_data[pj_data['Unnamed: 9']==s[2]][['Unnamed: 10','Unnamed: 11','Unnamed: 12']].values
    if s[2] in schools_id:
        school_id = schools_id[s[2]]
        page = requests.get(f'https://static.kaoyan.cn/json/school/{school_id}/info.json')
        ifo = json.loads(page.text)
        feature = ifo['data']['feature'][-1]
        gwurl = ifo['data']['site']['zhaoban_site']
    else:
        feature=''
        gwurl=''
    xx = list(xx[0]) if xx.size>0 else ['' for i in range(3)]
    pj = list(pj[0]) if pj.size>0 else ['']
    school_list = [feature,s[1]+s[2]]+pj+xx
    school_data = [school_list + sublist for sublist in school_data]
    school_data[0][-2]=gwurl
    zy_data+=school_data

col = ['分区','(代码)学校名称','学科评价','所在省','所在市','C9/985/211/省重点','考试方式','院系所','专业','研究方向','学习方式','考试科目一','考试科目二','考试科目三','考试科目四','考试参考书籍','拟招人数','学校官网','备注']
data = pd.DataFrame(zy_data,columns=col)
data.to_excel(f'{fxmc}-院校信息.xlsx',index=False)

In [114]:
data.to_excel(f'{fxmc}院校信息.xlsx',index=False)